# Dependenies
Let's download and import all the needed libraries.
- **PyOWM** is needed to retrieve the weather forecast,
- **DateTime** is needed to make arithmetics on dates,
- **Pandas** is needed to store training data in CSV format,
- **DeepPavlov** is choosed as a framework for this task.

In [0]:
%%capture
!pip install pyowm
!pip install deeppavlov

In [0]:
import pyowm
import datetime
import pandas as pd

from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

# Creation of intention dataset

For Russian language.

In [0]:
temp_intent_templates = [
    'какая погода {}',
    'какая температура {}',
    'какая будет температура {}',
    'сколько градусов {}',
    'сколько будет градусов {}',
    'тепло ли на улице {}',
    'холодно ли на улице {}',
    'тепло ли будет на улице {}',
    'холодно ли будет на улице {}',
]

wind_intent_templates = [
    'насколько ветрено {}',
    'насколько будет ветрено {}',
    'какой ветер {}',
    'сильно будет дуть {}',
]

rain_intent_templates = [
    'будет ли дождь {}',
    'какие осадки {}',
    'ждать ли ливень {}'
]

day_phrases = {
    '': 0,
    'сегодня': 0,
    'сейчас': 0,
    'завтра': 1,
    'через день': 1,
    'через один день': 1,
    'послезавтра': 2,
    'через два дня': 2,
    'через три дня': 3,
    'через 1 день': 1,
    'через 2 дня': 2,
    'через 3 дня': 3,
}

questions = []
intents = []

for template in temp_intent_templates:
    for phrase in day_phrases:
        questions.append(template.format(phrase))
        intents.append('temp-' + str(day_phrases[phrase]))
        
for template in wind_intent_templates:
    for phrase in day_phrases:
        questions.append(template.format(phrase))
        intents.append('wind-' + str(day_phrases[phrase]))

for template in rain_intent_templates:
    for phrase in day_phrases:
        questions.append(template.format(phrase))
        intents.append('rain-' + str(day_phrases[phrase]))

qa_dataset_ru = pd.DataFrame({'question': questions ,'intent': intents})
qa_dataset_ru.to_csv('qa_dataset_ru.csv', index=False, line_terminator='\n')

For English language.

In [0]:
temp_intent_templates = [
    'what is the weather like {}',
    'what is the temperature {}',
    'how may degrees {}',
    'is it cold {}',
    'is it cold outside {}',
    'is it warm {}',
    'is it warm outside {}',
]

wind_intent_templates = [
    'how windy {}',
    'what is the wind {}',
    'will it blow {}',
]

rain_intent_templates = [
    'is it going to rain {}',
    'will it rain {}',
    'what is the precipitation {}',
    'should i expect rain {}'
]

day_phrases = {
    '': 0,
    'today': 0,
    'now': 0,
    'tomorrow': 1,
    'in day': 1,
    'in one day': 1,
    'in two days': 2,
    'in three days': 3,
    'in 1 day': 1,
    'in 2 days': 2,
    'in 3 days': 3,
}

questions = []
intents = []

for template in temp_intent_templates:
    for phrase in day_phrases:
        questions.append(template.format(phrase))
        intents.append('temp-' + str(day_phrases[phrase]))
        
for template in wind_intent_templates:
    for phrase in day_phrases:
        questions.append(template.format(phrase))
        intents.append('wind-' + str(day_phrases[phrase]))

for template in rain_intent_templates:
    for phrase in day_phrases:
        questions.append(template.format(phrase))
        intents.append('rain-' + str(day_phrases[phrase]))

qa_dataset_en = pd.DataFrame({'question': questions ,'intent': intents})
qa_dataset_en.to_csv('qa_dataset_en.csv', index=False, line_terminator='\n')

# Chatbot

In [0]:
MY_LOCATION = 'Novosibirsk, RU'
OWM_API_KEY = 'b2a209b25b7a0486ee4c845b7e20d397'

DATAPATH = {
    'ru': 'qa_dataset_ru.csv',
    'en': 'qa_dataset_en.csv'
}

LOCALIZATION = {
    'ru': {
        'temp': 'Ожидается {0}°C при колебаниях между {1}°C и {2}°C.',
        'wind': 'Ожтдается ветер со скоростью {} м/с.',
        'rain': 'Будет {}.',
        'reject': 'Извиняюсь, но я не понял ваш вопрос.'
    },
    'en': {
        'temp': 'The temperature will be {0} °C flactuating between {1} °C and {2} °C.',
        'wind': 'The wind will be {} m/s.',
        'rain': 'It will be {} outside.',
        'reject': 'Sorry, I do not understand your question.'
    }
}

def create_intent_classifier(path):
    reject = PatternMatchingSkill(responses=['reject'], default_confidence=0.02)  
    intent = SimilarityMatchingSkill(data_path=path, x_col_name='question', y_col_name='intent',
                                     save_load_path=path[:-4] + '_model', config_type='tfidf_autofaq', train=True)
  
    return DefaultAgent([intent, reject], skills_selector=HighestConfidenceSelector())

class WeatherBot:
    def __init__(self, language='ru'):
        self.language = language
        self.classifier = create_intent_classifier(DATAPATH[language])
        self.answers = LOCALIZATION[language]
        self.owm = pyowm.OWM(OWM_API_KEY, language=language)
        
    def __call__(self, question):
        intent = self.classifier([question])[0]
        if intent == 'reject':
            return self.answers['reject']
        
        forecaster = self.owm.three_hours_forecast(MY_LOCATION)
        day = forecaster.when_starts('date') + datetime.timedelta(days=int(intent[5]))
        weather = forecaster.get_weather_at(day)

        if intent.startswith('temp'):
            temperature = weather.get_temperature(unit='celsius')
            return self.answers['temp'].format(temperature['temp'], temperature['temp_min'], temperature['temp_max'])
        elif intent.startswith('wind'):
            return self.answers['wind'].format(weather.get_wind(unit='meters_sec')['speed'])
        else:
            return self.answers['rain'].format(weather.get_status().lower())
        

# Chatting with WeatherBot

In [6]:
%%capture
bot_ru = WeatherBot('ru')
bot_en = WeatherBot('en')

2019-06-28 05:45:34.524 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/vectorizer/tfidf_vectorizer_ruwiki.pkl to /content/qa_dataset_ru_model/vectorizer/tfidf_vectorizer_ruwiki.pkl
2019-06-28 05:46:09.718 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/faq/school/tfidf_cos_sim_classifier.pkl to /content/qa_dataset_ru_model/faq/tfidf_cos_sim_classifier.pkl
2019-06-28 05:46:10.572 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 202: Loading model sklearn.feature_extraction.text:TfidfVectorizer from /content/qa_dataset_ru_model/vectorizer/tfidf_vectorizer_ruwiki.pkl
2019-06-28 05:46:13.77 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 209: Model sklearn.feature_extraction.textTfidfVectorizer loaded  with parameters
2019-06-28 05:46:13.78 WARNING in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component

In [7]:
questions = [
    'Сегодня будет дождь?',
    'Какая завтра температура?',
    'Послезавтра будет холодно?',
    'Будет ли ветрено через три дня?',
    'Когда Навальный станет прездиентом?'
]

for question in questions:
    print('В: {}\nО: {}\n'.format(question, bot_ru(question)))

В: Сегодня будет дождь?
О: Будет clouds.

В: Какая завтра температура?
О: Ожидается 18.17°C при колебаниях между 18.17°C и 18.17°C.

В: Послезавтра будет холодно?
О: Ожидается 19.55°C при колебаниях между 19.55°C и 19.55°C.

В: Будет ли ветрено через три дня?
О: Ожтдается ветер со скоростью 4.51 м/с.

В: Когда Навальный станет прездиентом?
О: Извиняюсь, но я не понял ваш вопрос.



In [8]:
questions = [
    'Will it rain tomorrow?',
    'How is the weather like today?',  
    'How strong wind will be in three days?', 
    'Who killed JFK?'
]

for question in questions:
    print('Q: {}\nA: {}\n'.format(question, bot_en(question)))

Q: Will it rain tomorrow?
A: It will be rain outside.

Q: How is the weather like today?
A: The temperature will be 17.51 °C flactuating between 17.05 °C and 17.51 °C.

Q: How strong wind will be in three days?
A: The wind will be 4.51 m/s.

Q: Who killed JFK?
A: Sorry, I do not understand your question.

